In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tools.standard_name_utils import optimize_parameters
from tools.common_utils import highlight_diff, read_jsonl

model_path = "../ChatGLM2-6B/ptuning/output/checkpoint-100"
top_p=0.7 
temperature=0.95

/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
model = model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/home/reymond/anaconda3/envs/afac2024/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [02:09<00:00, 43.25s/it]


In [3]:
response, history = model.chat(tokenizer, "你好", history=[], top_p=top_p, temperature=temperature)
print(response)

{"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["中集环科"]], "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "查询收盘价", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "查询收盘价", "required_parameters": ["api_0的结果", "昨日"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "3", "api_name": "减法计算", "required_parameters": ["api_2的结果", "api_1的结果"], "rely_apis": ["1", "2"], "tool_name": "数值计算"}], "result": ["api_3的结果"]}


In [4]:
json_file_path = 'data/dev.json'
data_list = read_jsonl(json_file_path)

df = pd.DataFrame(data_list)
df.head()

,input,output
0,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
1,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
2,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
3,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."
4,你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。...,"{""relevant APIs"": [{""api_id"": ""0"", ""api_name"":..."


In [5]:
data_stock = pd.read_excel('data/标准名.xlsx',sheet_name='股票标准名')
data_fund = pd.read_excel('data/标准名.xlsx',sheet_name='基金标准名')

fund_standard_name = data_fund['标准基金名称'].to_list()
stock_standard_name = data_stock['标准股票名称'].to_list()

In [6]:
stock_standard_name[:5]

['万马股份', '汉王科技', '珠江啤酒', '联络互动', '奇正藏药']

In [30]:
from random import random

total_eval_count = 0.0
correct_count = 0.0

for index, row in df.iterrows():
    if random() <= 0.9:
        continue
    
    input = row["input"]
    output = row["output"]
    response, history = model.chat(tokenizer=tokenizer, query=input, history=[], top_p=top_p, temperature=temperature)
    
    optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)
    
    total_eval_count += 1
    if optimized_resp == output:
        correct_count += 1
    else:
        print("-----data index-----")
        print(index)
        print("-----query input-----")
        print(input)
        print("-----output diff-----")
        print(highlight_diff(output, optimized_resp))
        print(optimized_resp)
        print()
    
    if total_eval_count == 100:
        break
    
print("预测正确的比例：" + f"{correct_count / total_eval_count :.2%}")

-----data index-----
19
-----query input-----
你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我想知道如果我3个月前买入100股海联金汇，然后今天卖出，我需要支付的印花税是多少。 
 query中提到的产品标准名可能是：前海开源股息率100强等权重股票型证券投资基金、前海开源强势共识100强等权重股票型证券投资基金、海联金汇、博道沪深300指数增强型证券投资基金C类、博道沪深300指数增强型证券投资基金A类、富国中证1000优选股票型证券投资基金C类、富国中证1000优选股票型证券投资基金A类、博道中证1000指数增强型证券投资基金C类、博道中证1000指数增强型证券投资基金A类、富国沪深300基本面精选股票型证券投资基金C类、富国沪深300基本面精选股票型证券投资基金A类、招商科技动力3个月滚动持有股票型证券投资基金C类、招商科技动力3个月滚动持有股票型证券投资基金A类、国泰君安中证1000优选股票型发起式证券投资基金C类、国泰君安中证1000优选股票型发起式证券投资基金A类、浙商汇金卓越优选3个月持有期股票型基金中基金(FOF)、同泰积极配置3个月持有期股票型基金中基金(FOF)C类、同泰积极配置3个月持有期股票型基金中基金(FOF)A类、大成中证360互联网+大数据100指数型证券投资基金C类、大成中证360互联网+大数据100指数型证券投资基金A类、长盛中证100指数证券投资基金、融通深证100指数证券投资基金、大成沪深300指数证券投资基金、华富中证100指数证券投资基金、银河定投宝中证腾讯济安价值100A股指数型发起式证券投资基金、银华中国梦30股票型证券投资基金、新疆前海联合海盈货币市场基金B类、新疆前海联合海盈货币市场基金A类、新疆前海联合汇盈货币市场基金B类、新疆前海联合汇盈货币市场基金A类、人保沪深300指数型证券投资基金、南方沪深300交易型开放式指数证券投资基金联接基金A类(后端)、长城消费30股票型证券投资基金C类、长城消费30股票型证券投资基金A类、诺德深证300指数分级证券投资基金、诺安中证100指数证券投资基金C类、诺安中证100指数证券投资基金A类、融通深证100指数证券投资基金C类、招商深证100指数证券投资基金C类、招商深证1

In [20]:
test_input = df.loc[35]["input"]
test_input

'你现在是一个金融领域专家，你需要通过编排api来得到用户query的答案，输出json格式。query是：我持有城地香江的1000股，假设我昨天全部卖出，和我今天全部卖出相比，我会亏掉多少钱 \n query中提到的产品标准名可能是：城地香江、富国中证1000优选股票型证券投资基金C类、富国中证1000优选股票型证券投资基金A类、景顺长城中证1000指数增强型证券投资基金C类、景顺长城中证1000指数增强型证券投资基金A类、国泰君安中证1000优选股票型发起式证券投资基金C类、国泰君安中证1000优选股票型发起式证券投资基金A类、华宝中证1000指数证券投资基金C类、华宝中证1000指数证券投资基金A类、香江控股、鹏华中证1000指数增强型证券投资基金C类、鹏华中证1000指数增强型证券投资基金A类、长信中证1000指数增强型证券投资基金C类、长信中证1000指数增强型证券投资基金A类、财通中证1000指数增强型证券投资基金C类、财通中证1000指数增强型证券投资基金A类、浙商中证1000指数增强型证券投资基金C类、浙商中证1000指数增强型证券投资基金A类、明亚中证1000指数增强型证券投资基金C类、明亚中证1000指数增强型证券投资基金A类、招商中证1000指数增强型证券投资基金C类、招商中证1000指数增强型证券投资基金A类、太平中证1000指数增强型证券投资基金C类、太平中证1000指数增强型证券投资基金A类、天弘中证1000指数增强型证券投资基金C类、天弘中证1000指数增强型证券投资基金A类、国金中证1000指数增强型证券投资基金C类、国金中证1000指数增强型证券投资基金A类、博道中证1000指数增强型证券投资基金C类、博道中证1000指数增强型证券投资基金A类、博时中证1000指数增强型证券投资基金C类、博时中证1000指数增强型证券投资基金A类、华安中证1000指数增强型证券投资基金C类、华安中证1000指数增强型证券投资基金A类、华夏中证1000指数增强型证券投资基金C类、华夏中证1000指数增强型证券投资基金A类、兴银中证1000指数增强型证券投资基金C类、兴银中证1000指数增强型证券投资基金A类、中银中证1000指数增强型证券投资基金C类、中银中证1000指数增强型证券投资基金A类、中欧中证1000指数增强型证券投资基金C类、中欧中证10

In [31]:
test_input = df.loc[19]["input"]
test_input += " 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价 请使用查询当前价"
response, history = model.chat(tokenizer=tokenizer, query=test_input, history=[], top_p=top_p, temperature=temperature)
optimized_resp = optimize_parameters(response, fund_standard_name, stock_standard_name)

output = df.loc[19]["output"]
print(highlight_diff(output, optimized_resp))
print(optimized_resp)

{"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["海联金汇"]], "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "查询当前价", "required_parameters": ["api_0的结果"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "乘法计算", "required_parameters": ["api_1的结果", "100"], "rely_apis": ["1"], "tool_name": "数值计算"}, {"api_id": "3", "api_name": "乘法计算", "required_parameters": ["api_2的结果", "0.001"], "rely_apis": ["2"], "tool_name": "数值计算"}], "result": ["api_3的结果"]}ame": "减法计算", "required_parameters": ["api_3的结果", "api_2的结果"], "rely_apis": ["3", "2"], "tool_name": "数值计算"}], "result": ["api_4的结果"]}
{"relevant APIs": [{"api_id": "0", "api_name": "查询代码", "required_parameters": [["海联金汇"]], "rely_apis": [], "tool_name": "股票查询"}, {"api_id": "1", "api_name": "查询收盘价", "required_parameters": ["api_0的结果", "今日"], "rely_apis": ["0"], "tool_name": "股票查询"}, {"api_id": "2", "api_name": "乘法计算", "required_parameters": ["api_1的结果", "100"], "rely_apis": ["1"],